# Cleaning of shark attacks data set

## Objetive

The main purpose of this project is double. In one hand, we want to know what shark species is la que más está registrada en los ataques. In the other hand, we want to determine the profile of the people who has provoked the incidents the most (nationality, activity, sex and age).
Global Shark Attack File defines a provoked incident as one in which the shark was speared, hooked, captured or in which a human drew "first blood"

In [1]:
#Imports
import pandas as pd
import numpy as np
import re

#Import data
data = pd.read_csv('../data/attacks.csv', encoding='latin-1')
print(data.shape)
data.head
data.columns


(25723, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

Our first task is to subset the initial dataset to keep only the variables that we are going to use:
* Type
* Country
* Activity
* Sex
* Age
* Species

In [2]:
# Subset the dataframe with the needed columns
df_subset = data[["Case Number", "Type", "Country", "Activity", "Sex ", "Age", "Species "]]

# Rename the Sex and Species columns to remove the last space and avoid problems in the future...
df_subset = df_subset.rename(columns={"Sex ": "Sex", "Species ": "Species"})

# Extract only the provoked types
df = df_subset[df_subset["Type"] == "Provoked"]

df

,Case Number,Type,Country,Activity,Sex,Age,Species
4,2018.06.04,Provoked,MEXICO,Free diving,M,NaN,"Tiger shark, 3m"
10,2018.05.24,Provoked,AUSTRALIA,Feeding sharks,M,32,Grey reef shark
14,2018.05.00,Provoked,AUSTRALIA,Feeding sharks,F,34,"Tawny nurse shark, 2m"
41,2018.02.17,Provoked,AUSTRALIA,Kayak fishing for sharks,M,34,NaN
55,2017.12.21,Provoked,MALAYSIA,Fishing / Wading,M,24,Juvenile nurse shark
...,...,...,...,...,...,...,...
6224,ND.0089,Provoked,VANUATU,Attempting to drive shark from area,M,NaN,A large hammerhead shark
6226,ND.0087,Provoked,USA,Skin diving. Grabbed shark's tail; shark turne...,M,19,"Nurse shark, 2.1 m [7']"
6250,ND.0055,Provoked,BAHAMAS,Testing movie camera in full diving dress,M,NaN,"Nurse shark, 2.1 m [7']"
6254,ND.0051,Provoked,CUBA,"Shark fishing, knocked overboard",M,50,NaN


In [3]:
#Evaluate the NA's
def evaluar_NA(data):
    # Pandas series denoting features and the sum of their null values
    null_sum = data.isna().sum()
    # Total
    total = null_sum.sort_values(ascending=False)
    # Percentage
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False) 
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])   
    return df_NA


## Retrato robot del bañista provocador

In [4]:
humans = df[["Type", "Country", "Activity", "Sex", "Age",]]
humans

,Type,Country,Activity,Sex,Age
4,Provoked,MEXICO,Free diving,M,NaN
10,Provoked,AUSTRALIA,Feeding sharks,M,32
14,Provoked,AUSTRALIA,Feeding sharks,F,34
41,Provoked,AUSTRALIA,Kayak fishing for sharks,M,34
55,Provoked,MALAYSIA,Fishing / Wading,M,24
...,...,...,...,...,...
6224,Provoked,VANUATU,Attempting to drive shark from area,M,NaN
6226,Provoked,USA,Skin diving. Grabbed shark's tail; shark turne...,M,19
6250,Provoked,BAHAMAS,Testing movie camera in full diving dress,M,NaN
6254,Provoked,CUBA,"Shark fishing, knocked overboard",M,50


In [5]:
# Evaluate the percentage of 
evaluar_NA(humans)

,Number of NA,Percent NA
Age,294,51.22
Sex,57,9.93
Activity,35,6.10
Country,3,0.52
Type,0,0.00


In [8]:
# Drop rows with na values
humans_cl = humans.dropna(axis=0)
humans_cl.shape

(262, 5)

Extract the activy

In [87]:
humans_cl["regActivity"] = humans_cl["Activity"].str.findall(r"(\w+\s+){0,1}\b(\w+ing)(\s\w+){0,2}")
humans_cl


/home/lee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Type,Country,Activity,Sex,Age,regActivity
10,Provoked,AUSTRALIA,Feeding sharks,M,32,"[(, Feeding, sharks)]"
14,Provoked,AUSTRALIA,Feeding sharks,F,34,"[(, Feeding, sharks)]"
41,Provoked,AUSTRALIA,Kayak fishing for sharks,M,34,"[(Kayak , fishing, sharks)]"
55,Provoked,MALAYSIA,Fishing / Wading,M,24,"[(, Fishing, ), (, Wading, )]"
104,Provoked,USA,Fishing,M,34,"[(, Fishing, )]"
...,...,...,...,...,...,...
5074,Provoked,AUSTRALIA,Standing in knee-deep water,M,19,"[(, Standing, knee)]"
5133,Provoked,NEW ZEALAND,Fishing,M,70,"[(, Fishing, )]"
5684,Provoked,AUSTRALIA,Fishing,M,21,"[(, Fishing, )]"
6226,Provoked,USA,Skin diving. Grabbed shark's tail; shark turne...,M,19,"[(Skin , diving, ), (began , towing, to)]"


In [83]:
# Cleaning
humans_cl["activity_cl"] = filter (lambda cityName: len (cityName) > 5, cities)


ValueError: Length of values does not match length of index

In [33]:
activity = humans_cl["Activity"].str.findall(r"\b(\w+\s){0,1}\b(\w+ing)\s\b(\w+){0,1}")
activity[0:50]

10            [(, Feeding, sharks)]
14            [(, Feeding, sharks)]
41         [(Kayak , fishing, for)]
55                               []
104                              []
122                              []
139               [(, Touching, a)]
143            [(, Attempting, to)]
196                              []
198                              []
237                              []
242                              []
251                              []
254                              []
262                              []
278                [(, Teasing, a)]
280                              []
285                              []
286                              []
366                              []
379                              []
386                              []
405                              []
444                              []
452                              []
468                              []
473                              []
481                         

In this part, we select the coluns the NULLs values from "Species" column. As we are only interested in provoked attaks, we first do a subset by type of attack.

In [ ]:
def

In [ ]:
def evaluar_NA(data):
    # Pandas series denoting features and the sum of their null values
    null_sum = data.isna().sum()
    # Total
    total = null_sum.sort_values(ascending=False)
    # Percentage
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False) 
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])   
    return df_NA

In [ ]:
evaluar_NA(df)

Because of the huge amount of NA values in shark species, we are going to split the data into two parts, one for analizar las especies y otro para los humanos

### Sharks attacks provoked by human by country

In [ ]:
d_sharks = df[["Country", "Species"]]

In [ ]:
# Drop "Species" rows with null values
species_clean = d_sharks["Species"].dropna(axis=0)
species_clean.shape

# Subset the dataframe with the indices of the not null "Species rows"
d_sharks_cl = d_sharks.loc[species_clean.index, ]
d_sharks_cl.shape

In [ ]:
# Check NAs
evaluar_NA(d_sharks_cl)

Extract the shark species

In [ ]:
#prueba
#d_sharks_cl['spp'] = d_sharks_cl["Species"].str.findall(r"\bshark")
# Extract from 1 up to 3 words before the word 'shark'
d_sharks_cl['spp'] = d_sharks_cl["Species"].str.findall(r"(?:[A-Za-z]+\s){1,3}[shark | catshark | Pporbeagle | pointer ]+")
print(d_sharks_cl)

In [ ]:
# Split the strings
d_sharks_cl['spp'] = d_sharks_cl["Species"].str.split()
print(d_sharks_cl['spp'])